# Прогнозирование с ARMA (пакет [`sktime`](https://www.sktime.net/en/stable/))

__Важно__: $ARMA$ рассматривается как частный случай $ARIMA$: 

$$
	ARMA(p,q)=ARIMA(p,0,q).
$$

In [ ]:
import numpy as np
import pandas as pd

from sktime.forecasting.arima import ARIMA, AutoARIMA
from sktime.utils.plotting import plot_series

import pandas_datareader.data as web

# настройки визуализация
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) недельные данные по Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity (Symbol [`WGS10YR`](https://fred.stlouisfed.org/series/WGS10YR)) с 2000-01-01 по 2023-12-31 и создадим датафрейм `rate`



In [ ]:
rate = web.DataReader(name='WGS10YR', data_source='fred', start='2000-01-01', end='2023-12-31')

Пусть `y` – первая разность ставки (не забудем удалить пропущенные наблюдения)

In [ ]:
y = rate.diff().dropna()

## Прогнозирование с ARMA

Построим прогноз для `y` модель ARMA(2,2) на 10 периодов

Спецификация

$$
	y_t=\alpha_0+\phi_1y_{t-1}+\phi_2y_{t-2}+u_1+\theta_1 u_{t-1}+\theta_2 u_{t-2}
$$

In [ ]:
forecaster = ARIMA(order=(2,0,2), with_intercept=True, trend=None)
forecaster.fit(y)

Численный прогноз

In [ ]:
y_pred = forecaster.predict(fh=np.arange(1, 11))
y_pred

# Можно делать сразу: подогнать и спрогнозировать
# y_pred = forecaster.fit_predict(y, fh=np.arange(1, 11))

In [ ]:
plot_series(y.tail(30), y_pred, labels=['y', 'y_pred'])

plt.show()

Доверительный интервал

In [ ]:
conf_int = forecaster.predict_interval(fh=np.arange(1, 11), coverage=0.9)
conf_int

In [ ]:
plot_series(y.tail(20), y_pred, labels=['y', 'y_pred'], markers=['o', 'X'], pred_interval=conf_int)

plt.show()

## Прогнозирование с ARMA (с трендом)

Построим прогноз для `y` модель ARMA(2,1) с трендом на 10 периодов

Спецификация

$$
	y_t=\alpha_0+\alpha_1 t+\phi_1y_{t-1}+\phi_2y_{t-2}+u_1+\theta u_{t-1}
$$

In [ ]:
forecaster = ARIMA(order=(2,0,2), with_intercept=True, trend='t')
y_pred = forecaster.fit_predict(y, fh=np.arange(1, 11))
y_pred

In [ ]:
plot_series(y.tail(30), y_pred, labels=['y', 'y_pred'])

plt.show()